<a href="https://colab.research.google.com/github/TarunSaxena1996/nlp/blob/main/lstm/fake_news_classifier_lstm_(krish_nayak).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification

In [1]:
import  pandas as pd

In [2]:
df=pd.read_csv('/content/WELFake_Dataset.csv')

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
len(df)

72134

In [5]:
###Drop Nan Values
df=df.dropna()

In [6]:
len(df)

71537

In [7]:
X=df.drop('label',axis=1)

In [8]:
X.shape

(71537, 3)

In [9]:
## Get the Dependent features
y=df['label']
y.shape

(71537,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.19.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [16]:
messages['title']

,title
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,SATAN 2: Russia unvelis an image of its terrif...
5,About Time! Christian Group Sues Amazon and SP...
...,...
72129,Russians steal research on Trump in hack of U....
72130,WATCH: Giuliani Demands That Democrats Apolog...
72131,Migrants Refuse To Leave Train At Refugee Camp...
72132,Trump tussle gives unpopular Mexican leader mu...


In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
messages.reset_index(inplace=True)

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm
ps = PorterStemmer()
corpus = []
for i in tqdm(range(0, len(messages))):

    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

print('completed')

100%|██████████| 71537/71537 [01:46<00:00, 672.47it/s]

completed


In [21]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2531, 1043, 4989, 160, 66, 1576, 3548, 2303, 320, 93, 1418, 1621],
 [3476,
  2975,
  4745,
  3864,
  1918,
  1137,
  1159,
  4561,
  4966,
  4002,
  4993,
  4069,
  1232,
  1621],
 [3368, 742, 425, 705, 1456, 4172, 300, 2827, 1001, 1659, 1495, 1727],
 [912, 4523, 478, 2179, 4122, 1648, 2523, 1395, 9, 4637, 4540],
 [4718, 300, 4500, 3727, 2496, 468, 2272, 1999, 4500],
 [2537, 4274, 1411, 2222, 1511, 2096, 3870, 1804, 1247, 488, 3339],
 [4386, 2706, 3824, 379, 4523, 1244, 4172, 3038, 2183, 1621],
 [1617,
  4918,
  85,
  3496,
  1909,
  4988,
  562,
  756,
  4007,
  1617,
  789,
  1745,
  2892,
  1626,
  1621],
 [2978, 957, 3059, 1952, 2448, 1271, 2141, 957],
 [4448, 1493, 4620, 1358, 322, 2053, 336, 3634],
 [38, 1121, 541, 3875, 1989, 356, 90, 3410, 90, 222],
 [667, 3666, 379, 1009, 221, 1995, 735, 4913, 4373],
 [1780, 3723, 4105, 3666, 4293, 3299, 1676, 4678, 1932, 1653, 1238],
 [3149, 2284, 4640, 1635, 2310, 1429, 2244, 4523, 1795, 3226],
 [2277, 888, 3044, 3629, 2396, 4386, 1258],
 

### Embedding Representation

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...   93 1418 1621]
 [   0    0    0 ... 4069 1232 1621]
 [   0    0    0 ... 1659 1495 1727]
 ...
 [   0    0    0 ... 2054 2447 1858]
 [   0    0    0 ... 2452 1692  769]
 [   0    0    0 ... 3616 4678 1946]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0, 2531, 1043, 4989,
        160,   66, 1576, 3548, 2303,  320,   93, 1418, 1621], dtype=int32)

## Creating model

In [25]:
embedding_vector_features=40

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [26]:
len(embedded_docs),y.shape

(71537, (71537,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((71537, 20), (71537,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#Train

In [32]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 35s 44ms/step - accuracy: 0.8080 - loss: 0.4028 - val_accuracy: 0.8868 - val_loss: 0.2610
Epoch 2/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 36s 37ms/step - accuracy: 0.9095 - loss: 0.2191 - val_accuracy: 0.8945 - val_loss: 0.2489
Epoch 3/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 42s 39ms/step - accuracy: 0.9264 - loss: 0.1849 - val_accuracy: 0.8937 - val_loss: 0.2527
Epoch 4/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.9367 - loss: 0.1587 - val_accuracy: 0.8925 - val_loss: 0.2718
Epoch 5/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 29s 39ms/step - accuracy: 0.9465 - loss: 0.1358 - val_accuracy: 0.8926 - val_loss: 0.2925
Epoch 6/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 29s 39ms/step - accuracy: 0.9549 - loss: 0.1161 - val_accuracy: 0.8925 - val_loss: 0.3007
Epoch 7/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.9623 - loss: 0.0998 - val_accuracy: 0.8905 - val_loss: 0.3594
Epoch 8/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - accuracy: 0.9680 - loss: 0.0823 - 

In [33]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

738/738 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step


array([[10111,  1582],
       [ 1116, 10799]])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8857167062012877

# adding dropouts

In [35]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [36]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 33s 40ms/step - accuracy: 0.8041 - loss: 0.4205 - val_accuracy: 0.8925 - val_loss: 0.2563
Epoch 2/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 40s 39ms/step - accuracy: 0.9055 - loss: 0.2273 - val_accuracy: 0.8937 - val_loss: 0.2516
Epoch 3/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 29s 39ms/step - accuracy: 0.9202 - loss: 0.1979 - val_accuracy: 0.8942 - val_loss: 0.2490
Epoch 4/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.9293 - loss: 0.1768 - val_accuracy: 0.8966 - val_loss: 0.2566
Epoch 5/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 39s 37ms/step - accuracy: 0.9371 - loss: 0.1576 - val_accuracy: 0.8936 - val_loss: 0.2627
Epoch 6/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.9421 - loss: 0.1464 - val_accuracy: 0.8930 - val_loss: 0.2732
Epoch 7/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 28s 38ms/step - accuracy: 0.9504 - loss: 0.1311 - val_accuracy: 0.8930 - val_loss: 0.2850
Epoch 8/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 28s 38ms/step - accuracy: 0.9537 - loss: 0.1212 - 

### Performance Metrics And Accuracy

In [41]:
y_pred_prob=model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
confusion_matrix(y_test,y_pred)

738/738 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step


array([[10332,  1361],
       [ 1172, 10743]])

In [42]:

accuracy_score(y_test,y_pred)

0.8927058624195188